In [1]:
import sys
sys.path.append("..")
import library

%load_ext line_profiler

In [2]:
# General Imports
import os
import cv2
import time
#import copy
#import mergedeep
import numpy as np
import pandas as pd

import torch
import ipywidgets
from PIL import Image
import matplotlib.pyplot as plt

# Display & Camera imports
from library.hardware import DispUtils
from library.hardware.DisplayGL import DisplayGL
from library.hardware.PylonCamera import PylonCamera

# Select GPU as device if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
%matplotlib inline

Using device: cuda


In [3]:
def bgr8_to_jpeg(frame, quality=75):
    return bytes(cv2.imencode('.jpg', frame, [cv2.IMWRITE_JPEG_QUALITY, quality])[1])

def update_widget(img_widget, value):
    img_widget.value = bgr8_to_jpeg(value)
    
img_widget = ipywidgets.Image(format='jpeg', value=bgr8_to_jpeg(np.zeros((1,1))), width=640, height=480)

**Setup Camera and SLM**

In [4]:
#Initializes the SLM
uD = DisplayGL(1, reverseX=False, reverseY=False)
slm = DisplayGL(2, reverseX=False, reverseY=False)

Using GLFW: 3.4.0 Wayland X11 GLX Null EGL OSMesa monotonic shared


In [5]:
cam_dev = PylonCamera('40308273')
cam_dev.configure(exposure=12000, trigLine="Line2", triggerDelay=2000, frameRate=120, reverseX=True, reverseY=False)
cam_dev.start(screens=[uD, slm])

Camera devices ID found: ['40308273']


**Calibration**

In [13]:
# Maybe make this into a dictionnary
img_size = (56, 56) ## image size on the Screen sent to uD
slm_size = (56, 56) ## size of the mask
pat_pad = (12, 12) ## padding next to each image, must be even numbers
cam_pad = (0, 0)  ## Padding to add when capturing imagelet with the camera
batch_layout = (8, 8) ## nb of images
batch_stacks = 64 ## nb images in the queue for displaying, too high: increase batch size, risk of filling memory + bad learning , too low: slower

Niter_slm = 4

In [14]:
%%time
#cam_dev.reset_screen_trigger()
cal_widget = ipywidgets.Image(format='jpeg', value=bgr8_to_jpeg(np.zeros((1,1))), width=600, height=450)
cal_widget2 = ipywidgets.Image(format='jpeg', value=bgr8_to_jpeg(np.zeros((1,1))), width=600, height=450)
train_wid = ipywidgets.HBox([cal_widget, cal_widget2])
display(train_wid)

cal_dict, slm_xy_offsets = DispUtils.calibrate_screens(screens=[uD, slm], # List of screens to calibrate, the first one is the reference
                                                       cam_dev=cam_dev,
                                                       pat_size=[img_size, slm_size], # List of images size for each screens
                                                       pat_pad=pat_pad,
                                                       cam_pad=cam_pad,
                                                       batch_layout=batch_layout,
                                                       n_iteration=Niter_slm,
                                                       update_widget=[lambda x: update_widget(cal_widget, x), lambda x: update_widget(cal_widget2, x)],
                                                       thres_lst=[120, 10], kern_lst=[7, 13],  capture_ref=True)

if(Niter_slm):
    DispUtils.display_calibration(cal_dict, update_widget=lambda x: update_widget(cal_widget, x), keys=['uD-out', 'slm-out'], disp_border=False)

Calibrating screen 0...
Calibrating screen 1 ...
{'slm-in_size': (52, 52), 'slm-out_size': (66, 68), 'padding': (12, 12), 'batch': (8, 8), 'uD-in_size': (56, 56), 'uD-out_size': (62, 66)}
CPU times: user 7.94 s, sys: 5.35 s, total: 13.3 s
Wall time: 7.35 s
